In [1]:
import sys
if '/home/ross/immunova' not in sys.path:
    sys.path.append('/home/ross/immunova')
from immunova.data.patient import Patient, Bug, Biology, Drug
from immunova.data.fcs_experiments import FCSExperiment
from immunova.data.utilities import get_fcs_file_paths
from immunova.data.mongo_setup import pd_init
import pandas as pd
import re
import os
pd_init()

In [4]:
patient_id = '175-09'
status = 'Stable'
n_ctrl_names = ['CD1c', 'HLA-DR']
t_ctrl_names = ['CXCR3', 'CD25', 'CD45RA', 'CCR7']
root = f'/media/ross/extdrive/PD_DS_Friendly/{status}/{patient_id}/'
get_fcs_file_paths(f'{root}PBMC_T', t_ctrl_names, 'FMO')

Warning! Multiple non-control (primary) files found in directory. Check before proceeding.


{'primary': ['/media/ross/extdrive/PD_DS_Friendly/Stable/175-09/PBMC_T/175-09 Stable PBMC T4 CD45RA_018.fcs',
  '/media/ross/extdrive/PD_DS_Friendly/Stable/175-09/PBMC_T/175-09 Stable PBMC T1_015.fcs'],
 'controls': [{'control_id': 'CXCR3',
   'path': '/media/ross/extdrive/PD_DS_Friendly/Stable/175-09/PBMC_T/175-09 Stable PBMC T2 FMO CXCR3_016.fcs'},
  {'control_id': 'CCR7',
   'path': '/media/ross/extdrive/PD_DS_Friendly/Stable/175-09/PBMC_T/175-09 Stable PBMC T3 FMO CCR7_017.fcs'}]}

In [23]:
def rename_tfiles(x, cell_origin):
    x = re.sub(f'{cell_origin}[\s_]+T2[\s_]+[0-9]{{3}}.fcs', 'PDMC T2 FMO CXCR3.fcs', x)
    x = re.sub(f'{cell_origin}[\s_]+T3[\s_]+[0-9]{{3}}.fcs', 'PDMC T3 FMO CD27.fcs', x)
    x = re.sub(f'{cell_origin}[\s_]+T4[\s_]+[0-9]{{3}}.fcs', 'PDMC T4 FMO CD45RA.fcs', x)
    x = re.sub(f'{cell_origin}[\s_]+T5[\s_]+[0-9]{{3}}.fcs', 'PDMC T5 FMO CCR7.fcs', x)
    return x

def rename_nfiles(x, cell_origin):
    x = re.sub(f'{cell_origin}[\s_]+N2[\s_]+[0-9]{{3}}.fcs', 'PDMC N2 FMO CD1c.fcs', x)
    x = re.sub(f'{cell_origin}[\s_]+N3[\s_]+[0-9]{{3}}.fcs', 'PDMC N3 FMO HLA-DR.fcs', x)
    return x

In [24]:
rename_nfiles('Peri 237-06 PDMC N1_001.fcs', 'PDMC')

'Peri 237-06 PDMC N1_001.fcs'

In [25]:
rename_nfiles('Peri 237-06 PDMC N2_002.fcs', 'PDMC')

'Peri 237-06 PDMC N2 FMO CD1c.fcs'

In [26]:
rename_tfiles('Peri 237-06 PDMC T1_001.fcs', 'PDMC')

'Peri 237-06 PDMC T1_001.fcs'

In [27]:
rename_tfiles('Peri 237-06 PDMC T4_004.fcs', 'PDMC')

'Peri 237-06 PDMC T4 FMO CD45RA.fcs'

In [35]:
def rename_duplicate(sdir, file, i=1):
    f = f'{i}_' + file
    if os.path.isfile(os.path.join(sdir, f)):
        i = i +1
        return rename_duplicate(sdir, file, i=i)
    return os.path.join(sdir, f)

In [38]:
root = '/media/ross/extdrive/PD_DS_Friendly/'
for subdir, dirs, files in os.walk(root):
    for file in files:
        new = None
        if os.path.basename(subdir) == 'PDMC_T':
            new = rename_tfiles(file, 'PDMC')
        if os.path.basename(subdir) == 'PBMC_T':
            new = rename_tfiles(file, 'PBMC')
        if os.path.basename(subdir) == 'PDMC_N':
            new = rename_nfiles(file, 'PDMC')
        if os.path.basename(subdir) == 'PBMC_N':
            new = rename_nfiles(file, 'PBMC')
        if new is not None:
            print(f'Renaming {file}...')
            if os.path.isfile(os.path.join(subdir, new)):
                os.rename(os.path.join(subdir, file), rename_duplicate(subdir, new))
            else:
                os.rename(os.path.join(subdir, file), os.path.join(subdir, new))

Renaming T Cell Panel_291-02 PDMC T1_019.fcs...
Renaming Stable PD 291-02 PDMC N1_013.fcs...
Renaming Stable 291-02 PBMC T2_015.fcs...
Renaming Stable 291-02 PBMC T1_014.fcs...
Renaming T Cell Panel_291-02 PBMC T4_017.fcs...
Renaming T Cell Panel_291-02 PBMC T3_016.fcs...
Renaming T Cell Panel_291-02 PBMC T5_018.fcs...
Renaming 305-02 Stable PDMC T2 FMO CXCR3_031.fcs...
Renaming 305-02 Stable PDMC T1_030.fcs...
Renaming 305-02 Stable PDMC T5 CD27_034.fcs...
Renaming 305-02 Stable PDMC T4 FMO CD45RA_033.fcs...
Renaming 305-02 Stable PDMC T3 FMO CCR7_032.fcs...
Renaming 305-02 Stable PDMC N2_020.fcs...
Renaming 305-02 Stable PDMC N1_019.fcs...
Renaming 305-02 Stable PDMC N3_021.fcs...
Renaming 305-02 Stable PBMC T4 FMO CD45RA_038.fcs...
Renaming 305-02 Stable PBMC T3 FMO CCR7_037.fcs...
Renaming 305-02 Stable PBMC T2 FMO CXCR3_036.fcs...
Renaming 305-02 Stable PBMC T5 FMO CD27_039.fcs...
Renaming 305-02 Stable PBMC T1_035.fcs...
Renaming 251-07 Stable PDMC T1_020.fcs...
Renaming 251-07 S

In [4]:
patient_id = '294-01'
status = 'Peritonitis'
n_ctrl_names = ['CD1c', 'HLA-DR']
t_ctrl_names = ['CXCR3', 'CD27', 'CD45RA', 'CCR7']
root = f'/media/ross/extdrive/PD_DS_Friendly/{status}/{patient_id}/'
get_fcs_file_paths(f'{root}PDMC_T', t_ctrl_names, 'FMO')

{'primary': ['/media/ross/extdrive/PD_DS_Friendly/Peritonitis/294-01/PDMC_T/1_Peri 294-01 PDMC 2 T Cells_T1_014.fcs'],
 'controls': [{'control_id': 'CXCR3',
   'path': '/media/ross/extdrive/PD_DS_Friendly/Peritonitis/294-01/PDMC_T/1_Peri 294-01 PDMC 2 T Cells_T2 FMO CXCR3_015.fcs'},
  {'control_id': 'CD27',
   'path': '/media/ross/extdrive/PD_DS_Friendly/Peritonitis/294-01/PDMC_T/1_Peri 294-01 PDMC 2 T Cells_T3 FMO CD27_016.fcs'},
  {'control_id': 'CD45RA',
   'path': '/media/ross/extdrive/PD_DS_Friendly/Peritonitis/294-01/PDMC_T/1_Peri 294-01 PDMC 2 T Cells_T4 FMO CD45RA_017.fcs'},
  {'control_id': 'CCR7',
   'path': '/media/ross/extdrive/PD_DS_Friendly/Peritonitis/294-01/PDMC_T/1_Peri 294-01 PDMC 2 T Cells_T5 FMO CCR7_018.fcs'}]}

In [6]:
from immunova.flow.readwrite. read_fcs import FCSFile

In [7]:
f = FCSFile('/media/ross/extdrive/PD_DS_Friendly/Peritonitis/294-01/PDMC_T/1_Peri 294-01 PDMC 2 T Cells_T1_014.fcs')

In [9]:
f.fluoro_mappings

[{'channel': 'FSC-A', 'marker': ''},
 {'channel': 'FSC-H', 'marker': ''},
 {'channel': 'SSC-A', 'marker': ''},
 {'channel': 'SSC-W', 'marker': ''},
 {'channel': 'Alexa Fluor 488-A', 'marker': 'CXCR3 FITC'},
 {'channel': 'Alexa Fluor 647-A', 'marker': 'CD161 APC'},
 {'channel': 'APC-Cy7-A', 'marker': 'CD3 APCFire750'},
 {'channel': 'Alexa Fluor 405-A', 'marker': 'CD27 BV421'},
 {'channel': 'AmCyan-A', 'marker': 'CD14-19-LD'},
 {'channel': 'BV605-A', 'marker': 'Va7-2'},
 {'channel': 'BV711-A', 'marker': 'CD8'},
 {'channel': 'PE-A', 'marker': 'Vd2'},
 {'channel': 'PE-CF594-A', 'marker': 'CD45RA PE Dazz'},
 {'channel': '7-AAD-A', 'marker': 'Pan gd PECy5'},
 {'channel': 'PE-Cy5-5-A', 'marker': 'CD4'},
 {'channel': 'PE-Cy7-A', 'marker': 'CCR7 rat'},
 {'channel': 'Time', 'marker': ''}]